# Custom Yarrowia Biomass Reaction
Update the biomass reaction with data from our strain

In [1]:
import pandas as pd
import cobra

### Load genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,10f7f79a0
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load 13C-MFA biomass reaction data

In [3]:
mfa_biomass_composition_df = pd.read_csv("../data/biomass_composition/yarrowia_mfa_biomass.csv")
mfa_biomass_composition_df

,metabolite,full_name,gsm_metabolite_id,mfa_coefficient_glucose,mfa_coefficient_oil
0,ALA,Alanine,ala_L[c],0.380,0.190
1,ARG,Arginine,arg_L[c],0.130,0.080
2,ASN,Asparagine,asn_L[c],0.196,0.196
3,ASP,Aspartate,asp_L[c],0.483,0.333
4,CYS,Cysteine,cys_L[c],0.036,0.036
5,GLN,Glutamine,gln_L[c],0.239,0.239
6,GLU,Glutamate,glu_L[c],0.520,0.290
7,GLY,Glycine,gly[c],0.350,0.230
8,HIS,Histidine,his_L[c],0.060,0.030
9,ILE,Isoleucine,ile_L[c],0.150,0.090


### Load GSM biomass metabolite classification

In [4]:
# load the biomass reaction
gsm_biomass_reaction_df = pd.read_csv("../data/biomass_composition/yarrowia_gsm_biomass.csv")
gsm_biomass_reaction_df

,Full Name,gsm_metabolite_id,metabolite_type
0,"1,3-beta-D-Glucan",13BDglcn[c],Carbohydrate
1,ADP,adp[c],Energy Molecule
2,L-Alanine,ala_L[c],Amino Acid
3,AMP,amp[c],RNA
4,L-Arginine,arg_L[c],Amino Acid
5,L-Asparagine,asn_L[c],Amino Acid
6,L-Aspartate,asp_L[c],Amino Acid
7,ATP,atp[c],Energy Molecule
8,Biomass,biomass[c],Other
9,Chitin Monomer,chitin[c],Carbohydrate


### Load MFA mmol C data

In [5]:
mfa_biomass_mmol_c_df = pd.read_csv("../data/biomass_composition/mfa_biomass_mmol_c.csv")
mfa_biomass_mmol_c_df

,type,glucose,oil
0,Amino Acid,18.607,12.157
1,Lipid,8.362,13.936
2,DNA,0.282,0.282
3,RNA,0.817,0.817
4,Carbohydrate,14.647,15.523
5,Total,42.715,42.715


### Determine the mmols of C from each precursor type in biomass reaction

In [6]:
def get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, biomass_reaction_id):
    total_mmols = 0

    biomass_reaction = model.reactions.get_by_id(biomass_reaction_id)

    for metabolite in biomass_reaction.metabolites:
        metabolite_id = metabolite.id
      
        # get the metabolite_type from 
        metabolite_type = gsm_biomass_reaction_df[gsm_biomass_reaction_df['gsm_metabolite_id'] == metabolite_id]['metabolite_type'].values[0]

        # get coefficient (reactants have negative coefficients)
        coefficient = -1 * biomass_reaction.metabolites[metabolite]
        
        # get mmol_c, extracting number of carbon atoms using cobrapy's API
        number_of_C = metabolite.elements.get('C', 0)

        if metabolite_type == type:
            # Update total_mmols
            total_mmols += coefficient * number_of_C

    return total_mmols

total_mmols = 0
for type in ['Amino Acid', 'Lipid', 'DNA', 'RNA', 'Carbohydrate']:
    
    mmol_C = get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, 'biomass_C')
    total_mmols += mmol_C
    print(f'{type}: {mmol_C}')

print(f'Total: {total_mmols}')

Amino Acid: 16.054480973000004
Lipid: 5.211946986999999
DNA: 0.390416315
RNA: 2.111885953
Carbohydrate: 14.104663089999999
Total: 37.873393318


### Add default, glucose, and oil columns to GSM biomass reaction df

In [11]:
default_coefficients = []
glucose_coefficients = []
oil_coefficients = []

c_limited_biomass_reaction = model.reactions.get_by_id("biomass_C")

# loop over the metabolites in the biomass reaction
for _, row in gsm_biomass_reaction_df.iterrows():
    
    metabolite_id = row.gsm_metabolite_id
    metabolite = model.metabolites.get_by_id(metabolite_id)

    # get the metabolite_type from 
    type = row.metabolite_type

    # get coefficient (reactants have negative coefficients)
    default_coefficient = -1 * c_limited_biomass_reaction.metabolites[metabolite]

    if type in ['Amino Acid', 'Lipid', 'DNA', 'RNA', 'Carbohydrate']:
        gsm_mmols = get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, 'biomass_C')

        # get scale factors
        glucose_mfa_mmols = mfa_biomass_mmol_c_df.loc[mfa_biomass_mmol_c_df['type'] == type, 'glucose'].values[0]
        oil_mfa_mmols = mfa_biomass_mmol_c_df.loc[mfa_biomass_mmol_c_df['type'] == type, 'oil'].values[0]  

        glucose_scale_factor = glucose_mfa_mmols / gsm_mmols
        oil_scale_factor = oil_mfa_mmols / gsm_mmols
    else:
        glucose_scale_factor = 1
        oil_scale_factor = 1
        
    default_coefficients.append(default_coefficient)
    glucose_coefficients.append(default_coefficient * glucose_scale_factor)
    oil_coefficients.append(default_coefficient * oil_scale_factor)

gsm_biomass_reaction_df['default_coefficient'] = default_coefficients
gsm_biomass_reaction_df['glucose_coefficient'] = glucose_coefficients
gsm_biomass_reaction_df['oil_coefficient'] = oil_coefficients

# Save the biomass reaction df to csv
gsm_biomass_reaction_df.to_csv("../results/biomass_composition/gsm_biomass_composition.csv", index=False)

gsm_biomass_reaction_df

,Full Name,gsm_metabolite_id,metabolite_type,default_coefficient,glucose_coefficient,oil_coefficient
0,"1,3-beta-D-Glucan",13BDglcn[c],Carbohydrate,0.943397,0.979671,1.038263
1,ADP,adp[c],Energy Molecule,-23.090000,-23.090000,-23.090000
2,L-Alanine,ala_L[c],Amino Acid,0.567939,0.658236,0.430063
3,AMP,amp[c],RNA,0.055401,0.021432,0.021432
4,L-Arginine,arg_L[c],Amino Acid,0.125563,0.145526,0.095081
5,L-Asparagine,asn_L[c],Amino Acid,0.186498,0.216150,0.141223
6,L-Aspartate,asp_L[c],Amino Acid,0.186531,0.216188,0.141248
7,ATP,atp[c],Energy Molecule,23.090000,23.090000,23.090000
8,Biomass,biomass[c],Other,-1.000000,-1.000000,-1.000000
9,Chitin Monomer,chitin[c],Carbohydrate,0.868358,0.901748,0.955679


### Create gluocse and oil biomass equations and add to model

In [8]:
def create_biomass_reaction(df, coefficient_column, reaction_id):
    # Create a new Reaction object
    biomass_reaction = cobra.Reaction(reaction_id)
    
    # For each row in the dataframe, add the metabolite and its coefficient to the reaction
    for _, row in df.iterrows():
        metabolite_id = row['gsm_metabolite_id']
        metabolite_coeff = -1 * row[coefficient_column]
        
        metabolite = model.metabolites.get_by_id(metabolite_id)
        
        # Add the metabolite to the reaction
        biomass_reaction.add_metabolites({metabolite: metabolite_coeff})
    
    return biomass_reaction

# Create the biomass reactions
glucose_biomass_reaction = create_biomass_reaction(gsm_biomass_reaction_df, 'glucose_coefficient', 'biomass_glucose')
oleic_acid_biomass_reaction = create_biomass_reaction(gsm_biomass_reaction_df, 'oil_coefficient', 'biomass_oil')

# add biomass reactions to model
model.add_reactions([glucose_biomass_reaction, oleic_acid_biomass_reaction])


### Check the mmols in the glucose and oil biomass reaction

In [9]:
# glucose
print('Default')
total_mmols = 0
for type in ['Amino Acid', 'Lipid', 'DNA', 'RNA', 'Carbohydrate']:
    
    mmol_C = get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, 'biomass_C')
    total_mmols += mmol_C
    print(f'{type}: {mmol_C}')


print(f'Total: {total_mmols}')
print()
print('Glucose')
total_mmols = 0
for type in ['Amino Acid', 'Lipid', 'DNA', 'RNA', 'Carbohydrate']:
    
    mmol_C = get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, 'biomass_glucose')
    total_mmols += mmol_C
    print(f'{type}: {mmol_C}')


print(f'Total: {total_mmols}')
print()
print('Oil')

# oil
total_mmols = 0
for type in ['Amino Acid', 'Lipid', 'DNA', 'RNA', 'Carbohydrate']:
    
    mmol_C = get_gsm_mmol_c(model, gsm_biomass_reaction_df, type, 'biomass_oil')
    total_mmols += mmol_C
    print(f'{type}: {mmol_C}')


print(f'Total: {total_mmols}')

Default
Amino Acid: 16.054480973000004
Lipid: 5.211946986999999
DNA: 0.390416315
RNA: 2.111885953
Carbohydrate: 14.104663089999999
Total: 37.873393318

Glucose
Amino Acid: 18.606999999999996
Lipid: 8.362
DNA: 0.282
RNA: 0.817
Carbohydrate: 14.647
Total: 42.714999999999996

Oil
Amino Acid: 12.156999999999995
Lipid: 13.936000000000003
DNA: 0.282
RNA: 0.817
Carbohydrate: 15.523000000000001
Total: 42.714999999999996


### Save the updated model

In [10]:
# save model
cobra.io.save_json_model(model, "../genome_scale_models/iYLI647_corr_3.json")